In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from IPython.display import HTML
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np
import os

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=32

In [ ]:
# Load the dataset using image_dataset_from_directory
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "beansLeaf",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
class_names

In [ ]:
# Convert dataset to NumPy arrays
images = []
labels = []
for image_batch, label_batch in dataset:
    images.append(image_batch.numpy())
    labels.append(label_batch.numpy())
images = np.concatenate(images)
labels = np.concatenate(labels)

In [ ]:
# Create an instance of the ImageDataGenerator with desired augmentation options
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    vertical_flip=False,
    fill_mode='nearest'
)

In [ ]:
# Create the directory if it doesn't exist
augmented_images_dir = "augmented_images"
if not os.path.exists(augmented_images_dir):
    os.makedirs(augmented_images_dir)


In [ ]:
# Generate augmented images and save them to the directory
augmented_data_generator = data_augmentation.flow(images, labels, batch_size=BATCH_SIZE, shuffle=True)
for i, (augmented_images, augmented_labels) in enumerate(augmented_data_generator):
    for j, augmented_image in enumerate(augmented_images):
        save_path = os.path.join(augmented_images_dir, f"augmented_image_{i * BATCH_SIZE + j}.png")
        tf.keras.preprocessing.image.save_img(save_path, augmented_image)
    if (i + 1) * BATCH_SIZE >= len(images):
        break


In [ ]:
# Create a generator function for augmented image batches and labels
def augmented_data_generator():
    for image_batch, label_batch in data_augmentation.flow(images, labels, batch_size=BATCH_SIZE, shuffle=False):
        yield image_batch, label_batch


In [ ]:
# Create a new dataset from the generator function
augmented_dataset = tf.data.Dataset.from_generator(
    augmented_data_generator,
    output_signature=(
        tf.TensorSpec(shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(BATCH_SIZE,), dtype=tf.int32)
    )
)

In [ ]:
# Concatenate the augmented dataset and custom dataset
combined_dataset = augmented_dataset.concatenate(dataset)

In [ ]:
# Shuffle and batch the combined dataset
combined_dataset = combined_dataset.shuffle(len(images)).batch(BATCH_SIZE)

In [ ]:
# Split the combined dataset into training and validation sets
val_size = int(0.2 * len(images))
train_dataset = combined_dataset.skip(val_size)
val_dataset = combined_dataset.take(val_size)

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

In [ ]:
# len(dataset)

In [ ]:
# train_size = 0.8
# len(dataset)*train_size

In [ ]:
# train_ds = dataset.take(54)
# len(train_ds)


In [ ]:
# test_ds = dataset.skip(54)
# len(test_ds)
# val_size=0.1
# len(dataset)*val_size


In [ ]:
# val_ds = test_ds.take(6)
# len(val_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Creating a Layer for Resizing and Normalization
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Applying Data Augmentation to Train Dataset
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:


# Model Architecture
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)
model.summary()


In [ ]:
# Compiling the Model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

In [ ]:
# scores = model.evaluate(test_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
print(len(acc))
print(len(val_acc))


In [ ]:
# prediction on a sample image

import numpy as np
for images_batch, labels_batch in test_ds.take(2):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[8])])

In [ ]:
# function for inference
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
# import os
# model_version=max([int(i) for i in os.listdir("./models") + [0]])+1
# model.save(f"./models/{model_version}")

In [ ]:
model.save("../beansmodel.h5")